In [1]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import imageio
import pandas as pd
# from data_prep_gasflow import features, targets, features_test, targets_test
from torchsummary import summary

In [2]:
# path = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/checkpoint_ohneneu_16dim.pth.tar'
# lädt trainiertes Modell ein
# path = 'checkp_xy_mit_16dim.pth.tar'
path = 'checkpoint_xy_16dim_101920.pth.tar' # relative path

In [3]:
# map_location=torch.device('cpu')
checkpoint = torch.load(path,  map_location=lambda storage, loc: storage)
model_structure = print(checkpoint['model'])


Sequential(
  (0): Linear(in_features=4, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): Linear(in_features=256, out_features=128, bias=True)
  (9): ReLU()
  (10): Linear(in_features=128, out_features=64, bias=True)
  (11): ReLU()
  (12): Linear(in_features=64, out_features=64, bias=True)
  (13): ReLU()
  (14): Linear(in_features=64, out_features=32, bias=True)
  (15): ReLU()
  (16): Linear(in_features=32, out_features=16, bias=True)
  (17): ReLU()
  (18): Linear(in_features=16, out_features=16, bias=True)
)


In [4]:
# baue das Netz nach / geht ggf. auch auf anderem Weg
net = torch.nn.Sequential(
        torch.nn.Linear(4, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 512),
        torch.nn.ReLU(),
        torch.nn.Linear(512, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 256),
        torch.nn.ReLU(),
        torch.nn.Linear(256, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 16),
        torch.nn.ReLU(),
        torch.nn.Linear(16, 16),
    )

In [5]:
model = net
model.load_state_dict(checkpoint['state_dict'])
model.eval()
summary(model, (2,4)) # 2 Dimensionen und 4 Inputs

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 2, 512]           2,560
              ReLU-2               [-1, 2, 512]               0
            Linear-3               [-1, 2, 512]         262,656
              ReLU-4               [-1, 2, 512]               0
            Linear-5               [-1, 2, 256]         131,328
              ReLU-6               [-1, 2, 256]               0
            Linear-7               [-1, 2, 256]          65,792
              ReLU-8               [-1, 2, 256]               0
            Linear-9               [-1, 2, 128]          32,896
             ReLU-10               [-1, 2, 128]               0
           Linear-11                [-1, 2, 64]           8,256
             ReLU-12                [-1, 2, 64]               0
           Linear-13                [-1, 2, 64]           4,160
             ReLU-14                [-1

In [6]:
# data_raw= pd.read_csv('C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/gasflowdata_complex_16dim.csv')
data_raw= pd.read_csv('gasflowdata_mitA_16dim.csv')

In [7]:
gas = data_raw
for ii in range(16):
    gas = gas.drop(['time_'+str(ii+1)], axis=1)
#     print(ii)
    
gas.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16,carbohydrates,proteins,lipids,dilution
0,1691.696473,0.194330,0.027472,0.020199,0.019492,0.019414,0.019405,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.019404,0.814724,0.079125,0.106152,0.133840
1,949.997750,0.038927,0.013907,0.013027,0.012980,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.012977,0.905792,0.090004,0.004204,0.179608
2,2507.140873,1.407618,0.086091,0.034676,0.027808,0.026578,0.026338,0.026290,0.026280,0.026278,0.026277,0.026277,0.026277,0.026277,0.026277,0.026277,0.126987,0.632277,0.240736,0.095542
3,1002.798536,0.045099,0.014782,0.013675,0.013611,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.013607,0.913376,0.050319,0.036305,0.174718
4,1316.125423,0.090699,0.019329,0.016368,0.016138,0.016119,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.016117,0.632359,0.198621,0.169020,0.152592


In [8]:
for field in ['dilution']:
# for field in ['dilution', 'Gasflow_end']:
    max_field = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field

In [9]:
gasflow_list = []
for ii in range(16):
    gasflow_list.append('Gasflow_' + str(ii+1))

print(gasflow_list)

['Gasflow_1', 'Gasflow_2', 'Gasflow_3', 'Gasflow_4', 'Gasflow_5', 'Gasflow_6', 'Gasflow_7', 'Gasflow_8', 'Gasflow_9', 'Gasflow_10', 'Gasflow_11', 'Gasflow_12', 'Gasflow_13', 'Gasflow_14', 'Gasflow_15', 'Gasflow_16']


In [10]:
for field in [gasflow_list]:
# for field in ['dilution', 'Gasflow_end']:
    max_field = gas[field].max()
    gas.loc[:,field] = (gas[field])/max_field

gas.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16,carbohydrates,proteins,lipids,dilution
0,0.645230,0.000069,0.000010,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.814724,0.079125,0.106152,0.669218
1,0.362338,0.000014,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.905792,0.090004,0.004204,0.898064
2,0.956248,0.000497,0.000030,0.000014,0.000011,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.000010,0.126987,0.632277,0.240736,0.477721
3,0.382477,0.000016,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.913376,0.050319,0.036305,0.873614
4,0.501983,0.000032,0.000007,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.632359,0.198621,0.169020,0.762983


In [11]:
torch.manual_seed(0)  

In [12]:
sample = np.random.choice(gas.index, size=int(len(gas)*0.9), replace=False)
data_raw_1, test_data_1 = gas.loc[sample], gas.drop(sample)

# # split into features and targets
columns = gasflow_list

features, targets = data_raw_1.drop(columns, axis=1), data_raw_1[columns]
features_test, targets_test = test_data_1.drop(columns, axis=1), test_data_1[columns]

In [13]:
features_test.head()

,carbohydrates,proteins,lipids,dilution
4,0.632359,0.198621,0.169020,0.762983
6,0.278498,0.003628,0.717873,0.626336
12,0.957167,0.042710,0.000123,0.066237
44,0.186873,0.561925,0.251202,0.164472
50,0.276025,0.462288,0.261687,0.323354


In [14]:
targets_test.head()

,Gasflow_1,Gasflow_2,Gasflow_3,Gasflow_4,Gasflow_5,Gasflow_6,Gasflow_7,Gasflow_8,Gasflow_9,Gasflow_10,Gasflow_11,Gasflow_12,Gasflow_13,Gasflow_14,Gasflow_15,Gasflow_16
4,0.501983,0.000032,0.000007,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006,0.000006
6,0.715345,0.000110,0.000013,0.000010,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
12,0.134650,0.281225,0.277501,0.308227,0.308601,0.308890,0.309091,0.309226,0.309313,0.309370,0.309394,0.309439,0.309458,0.309465,0.309468,0.309469
44,0.226340,0.604045,0.606285,0.673972,0.674746,0.675154,0.675361,0.675465,0.675516,0.675540,0.675548,0.675560,0.675562,0.675563,0.675563,0.675563
50,0.505059,0.008643,0.000255,0.000053,0.000024,0.000017,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015,0.000015


In [15]:
# x_data = torch.tensor(features.values, dtype=torch.float)
# y_data = torch.tensor(targets.values, dtype=torch.float)

# x_test = torch.tensor(features_test.values, dtype=torch.float)
# y_test = torch.tensor(targets_test.values, dtype=torch.float)

x_data = checkpoint['x_data']
y_data = checkpoint['y_data']
x_test = checkpoint['x_test']
y_test = checkpoint['y_test']

In [16]:
def my_loss(output, target):
    loss = ((output - target)**2).sum()
    return loss

In [17]:
prediction_test = net(x_test)

In [18]:
print(y_test.shape, prediction_test.shape)

torch.Size([1200, 16]) torch.Size([1200, 16])


In [19]:
# torch.subtract(y_test, prediction_test)


In [24]:
time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]
t = time_series.loc[0].to_numpy()

# plt.scatter(t, y_test[1,:])
# plt.scatter(t, prediction_test[1,:].detach().numpy())

score_model_prediction = 100 - (abs((prediction_test.detach().numpy()-y_test.detach().numpy())/y_test.detach().numpy()))
score_model = (np.mean(score_model_prediction, 0))

plt.scatter(t, score_model)
# # plt.scatter(t,score_model)
# # score_complete = np.mean(score_model)
print( min(score_model), score_model_prediction)

9.175159 [[ 99.99145   68.9047    52.63571  ...  78.767525 -38.531387 -35.756073]
 [ 99.99357   98.575386  89.62614  ...  96.385185  79.93657   91.66512 ]
 [ 99.9957    99.4831    81.587006 ...  79.21475   86.43692   73.48992 ]
 ...
 [ 99.99997   96.32153   97.00931  ...  75.80441   72.66248   87.30686 ]
 [ 99.999626  93.79736   78.462234 ...  82.864426  69.95236   95.202156]
 [ 99.9887    99.99626   99.99871  ...  99.998566  99.999344  99.999   ]]


In [25]:
print(features_test)

       carbohydrates  proteins    lipids  dilution
4           0.632359  0.198621  0.169020  0.762983
6           0.278498  0.003628  0.717873  0.626336
12          0.957167  0.042710  0.000123  0.066237
44          0.186873  0.561925  0.251202  0.164472
50          0.276025  0.462288  0.261687  0.323354
...              ...       ...       ...       ...
11917       0.240571  0.003771  0.755658  0.942768
11973       0.036966  0.078289  0.884745  0.778451
11974       0.054108  0.040654  0.905238  0.301825
11980       0.043555  0.490665  0.465780  0.367943
11995       0.474006  0.508674  0.017321  0.338530

[1200 rows x 4 columns]


## Find optimal dilution at specific substrate values

In [26]:
x_new_1 = x_test
print(x_new_1)

tensor([[0.9058, 0.0900, 0.0042, 0.8981],
        [0.4854, 0.1172, 0.3975, 0.6431],
        [0.1419, 0.5509, 0.3072, 0.5305],
        ...,
        [0.1525, 0.1712, 0.6763, 0.6391],
        [0.1445, 0.2327, 0.6228, 0.6763],
        [0.5953, 0.3500, 0.0547, 0.1453]])


In [27]:
# Erstelle Liste mit möglichen Dilutionen bei Substrateingang

dil_list = []
test_list = []
x_new = []
x_t = np.zeros((100,4))
# print(x_t)
for iii in range(100):
    dil_list.append((iii+1)/100)
    x_new = x_new_1[0].numpy()
    x_new[3] = (iii+1)/100
    test_list.append(x_new[:])
#     print(x_new)
    x_t[:][iii] = x_new 

In [28]:
print(x_t)

[[0.90579194 0.09000376 0.00420431 0.01      ]
 [0.90579194 0.09000376 0.00420431 0.02      ]
 [0.90579194 0.09000376 0.00420431 0.03      ]
 [0.90579194 0.09000376 0.00420431 0.04      ]
 [0.90579194 0.09000376 0.00420431 0.05      ]
 [0.90579194 0.09000376 0.00420431 0.06      ]
 [0.90579194 0.09000376 0.00420431 0.07      ]
 [0.90579194 0.09000376 0.00420431 0.08      ]
 [0.90579194 0.09000376 0.00420431 0.09      ]
 [0.90579194 0.09000376 0.00420431 0.1       ]
 [0.90579194 0.09000376 0.00420431 0.11      ]
 [0.90579194 0.09000376 0.00420431 0.12      ]
 [0.90579194 0.09000376 0.00420431 0.13      ]
 [0.90579194 0.09000376 0.00420431 0.14      ]
 [0.90579194 0.09000376 0.00420431 0.15000001]
 [0.90579194 0.09000376 0.00420431 0.16      ]
 [0.90579194 0.09000376 0.00420431 0.17      ]
 [0.90579194 0.09000376 0.00420431 0.18000001]
 [0.90579194 0.09000376 0.00420431 0.19      ]
 [0.90579194 0.09000376 0.00420431 0.2       ]
 [0.90579194 0.09000376 0.00420431 0.20999999]
 [0.90579194 

In [29]:
# Wandle Liste von Listen in Tensorwerte um
x_test_new = torch.tensor(x_t, dtype=torch.float)

In [30]:
# Nutze das Netz um Vorhersagen für die verschiedenen Dilutionen bei Substratwerten zu machen
pred_sample = net(x_test_new)
pred_sample = pred_sample.detach().numpy()
print(pred_sample)
pred_sample.shape

# schreibe die Ergebnisse der Methangasmenge in csv-file
import csv

with open("out.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(pred_sample)

[[ 1.18428469e-01  1.74012735e-01  1.49622232e-01 ...  1.65893808e-01
   1.65672556e-01  1.65524840e-01]
 [ 1.18593693e-01  1.91571325e-01  1.64370343e-01 ...  1.83513120e-01
   1.82345554e-01  1.82713702e-01]
 [ 1.16401136e-01  2.08873853e-01  1.80243686e-01 ...  2.02513471e-01
   2.00313479e-01  2.01264441e-01]
 ...
 [ 2.97888577e-01 -7.95304775e-04  5.34065068e-04 ... -1.40771270e-04
   6.61551952e-04  9.13256314e-04]
 [ 2.90513456e-01 -5.59985638e-04  3.38166952e-04 ... -5.92768192e-04
   8.21202993e-04  1.17542641e-03]
 [ 2.83157527e-01 -1.39102340e-04  1.05630606e-05 ... -1.22572482e-03
   9.72807407e-04  1.46427378e-03]]


In [31]:
# bestimme den maximalen Gasstrom im Gleichgewicht

def max_value(pred_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
#     print(list_1)
    return max([sublist[-1] for sublist in pred_sample])

print(max_value(pred_sample))
max_sample = max_value(pred_sample)

# bestimme den zugehörigen Index

def index_finding(pred_sample, max_sample):
    list_1 = [sublist[-1] for sublist in pred_sample]
    indices = [i for i, x in enumerate(list_1) if x == max_sample]
    return indices

print(index_finding(pred_sample, max_sample))

0.9759967
[25]


In [32]:
# Gegenprobe

x_point = torch.tensor(x_t[:][4], dtype=torch.float)
pred_sample_point = net(x_point)
pred_sample_point = pred_sample_point.detach().numpy()
print(pred_sample_point)

[0.12202275 0.2390291  0.21742475 0.23892364 0.24173096 0.2413174
 0.2414518  0.243189   0.24512702 0.24380425 0.24385314 0.2430589
 0.24416153 0.24456182 0.24331209 0.24394067]


In [33]:
carbo = 0.2
protein = 0.25
inert_x = 0.1
inert_s = 0.2
fat = 0.25

dilution = 0.01

In [34]:
model_ADM = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/Netze_Regressionen_ASIM/ADM_model/ADM_Sim.m'

In [35]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [36]:
eng.model_ADM(inert_s, inert_x, carbo, protein, fat, dilution)

MatlabExecutionError: Undefined function 'model_ADM' for input arguments of type 'double'.


In [37]:
%%file multi_lin_func.m

function p = multi_lin_func(a,b)
    % in-file comments can be added like this
    a = a+1;
    b = b+1;
    p = a+b;
end

Writing multi_lin_func.m


In [38]:
multi_lin_func(1,2)

NameError: name 'multi_lin_func' is not defined

# ab hier normaler Code

In [39]:
# Testing a sample of testdata
pred_sample = net(x_test[0])
pred_sample = pred_sample.detach().numpy()
y_real = y_test[0].detach().numpy()

print(x_test[0])
print(pred_sample)
print(y_real)


# showing deviation
print(y_real - pred_sample)


time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]
t = time_series.loc[0].to_numpy()

plt.scatter(t, y_real)
plt.scatter(t, pred_sample)
plt.show()

tensor([0.9058, 0.0900, 0.0042, 1.0000])
[ 2.8315735e-01 -1.3916194e-04  1.0633841e-05  7.7040866e-05
 -3.0832458e-04  1.1470914e-04  8.0202520e-04  4.1509420e-04
  6.1902776e-04  3.2304600e-04  7.3873997e-04 -7.5429678e-05
 -3.9717369e-04 -1.2257844e-03  9.7286329e-04  1.4643092e-03]
[3.6233848e-01 1.3744505e-05 4.8880670e-06 5.0723061e-06 5.0512426e-06
 5.0492622e-06 5.0488688e-06 5.0487488e-06 5.0487088e-06 5.0486951e-06
 5.0486919e-06 5.0486888e-06 5.0486883e-06 5.0486883e-06 5.0486883e-06
 5.0486883e-06]
[ 7.9181135e-02  1.5290645e-04 -5.7457742e-06 -7.1968563e-05
  3.1337581e-04 -1.0965988e-04 -7.9697632e-04 -4.1004544e-04
 -6.1397906e-04 -3.1799730e-04 -7.3369127e-04  8.0478363e-05
  4.0222239e-04  1.2308331e-03 -9.6781459e-04 -1.4592605e-03]


In [40]:
loss = my_loss(prediction_test, y_test)
print(loss)

tensor(16.3405, grad_fn=<SumBackward0>)


In [41]:
score_model_pre = 100-(abs((prediction_test - y_test)/y_test))
score_model = torch.mean(score_model_pre)
print(score_model)

prediction_train = net(x_data)
score_model_pre = 100-(abs((prediction_train - y_data)/y_data))
score_model = torch.mean(score_model_pre)
print(score_model)

print(prediction_test.detach().numpy())
print(y_test.detach().numpy())

abw = ((prediction_test.detach().numpy() - y_test.detach().numpy())**2)
print(abw.sum())
print(abw.shape)

max_list = []
for ii in range(16):
    max_list.append(max(abw[ii,:]))
print(max_list)

tensor(42.9936, grad_fn=<MeanBackward0>)
tensor(77.6101, grad_fn=<MeanBackward0>)
[[ 3.5924011e-01  4.4113398e-04 -2.2663176e-04 ... -1.0214746e-04
  -6.9435313e-04 -6.8034139e-04]
 [ 6.9100136e-01 -3.7774444e-05 -1.0450557e-04 ... -2.1308661e-05
   1.7165020e-04  7.6071825e-05]
 [ 8.6374545e-01  3.9832294e-04  3.8761273e-04 ... -1.8543005e-04
   1.3648719e-04  2.5782827e-04]
 ...
 [ 6.9178963e-01 -2.5288761e-04 -2.3167580e-05 ... -1.9505620e-04
   2.3829564e-04  1.1514826e-04]
 [ 6.3155508e-01 -3.4558773e-04 -1.9850582e-04 ... -1.2390316e-04
   2.3841113e-04  4.4520944e-05]
 [ 2.0869133e-01  5.4203510e-01  5.4702932e-01 ...  6.0854405e-01
   6.0902202e-01  6.0880953e-01]]
[[3.62338483e-01 1.37445049e-05 4.88806700e-06 ... 5.04868831e-06
  5.04868831e-06 5.04868831e-06]
 [6.86587334e-01 8.89612711e-05 1.11486106e-05 ... 8.14920259e-06
  8.14920259e-06 8.14920259e-06]
 [8.67476821e-01 2.62590096e-04 1.99666647e-05 ... 9.37213827e-06
  9.37213827e-06 9.37213827e-06]
 ...
 [6.91766858e-01

In [42]:
import numpy as np
import numpy.matlib
# import mplcursors

time_list = []
for ii in range(16):
    time_list.append('time_' + str(ii+1))

time_series = data_raw[time_list]


t = time_series.loc[0].to_numpy()
time_series.head()

,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,time_9,time_10,time_11,time_12,time_13,time_14,time_15,time_16
0,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
1,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
2,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
3,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300
4,3.030303,18.181818,33.333333,48.484848,63.636364,78.787879,93.939394,109.090909,124.242424,139.393939,148.484848,178.787879,209.090909,239.393939,269.69697,300


In [43]:
%matplotlib qt

In [44]:


from mpl_toolkits.mplot3d import Axes3D
y1 = y_test.numpy()
y2 = prediction_test.detach().numpy()
t = time_series.loc[0].to_numpy()
# t = np.matlib.repmat(t, len(y1), 1)
print(y1[0,3])
print(y2[0,3])

for i in range(70,80):
    fig = plt.figure()
    plt.plot(t, y1[i+10,:], 'bo--', label=r'$\dot m(CH_4)$ ADM1' , alpha=0.5)
    plt.plot(t, y2[i+10,:], marker=6, color='r', label=r'$\dot m(CH_4)$ DNN', alpha=0.5)
    plt.legend(loc='best')
    plt.xlabel('Zeit [d]')
    plt.ylabel(r'$\dot m(CH_4)(skaliert)$')
    plt.show()

5.072306e-06
-0.00011156686


In [69]:
# path = 'C:/Users/david.wagner/Desktop/Biomasse-Institut/Machine_Learning_Corr/checkpoint_ohneAuswasch_16dim_1.pth.tar'
# checkpoint = {'x_test': x_test, 'y_test': y_test, 'x_data': x_data, 'y_data': y_data,
#               'model': model,
#               'state_dict': model.state_dict()}

# torch.save(checkpoint, path)

In [70]:
# checkpoint_1 = torch.load(path,  map_location=lambda storage, loc: storage)

In [71]:
# x_data_1 = checkpoint['x_data'] 

In [73]:
# print(x_data_1, x_data)

tensor([[0.4294, 0.0866, 0.4840, 1.0000],
        [0.4852, 0.3456, 0.1692, 0.9925],
        [0.4716, 0.4852, 0.0433, 0.5376],
        ...,
        [0.6208, 0.3367, 0.0425, 0.4188],
        [0.3414, 0.5566, 0.1020, 0.5719],
        [0.6203, 0.2340, 0.1457, 0.9593]]) tensor([[0.4294, 0.0866, 0.4840, 1.0000],
        [0.4852, 0.3456, 0.1692, 0.9925],
        [0.4716, 0.4852, 0.0433, 0.5376],
        ...,
        [0.6208, 0.3367, 0.0425, 0.4188],
        [0.3414, 0.5566, 0.1020, 0.5719],
        [0.6203, 0.2340, 0.1457, 0.9593]])
